In [27]:
import numpy as np
import matplotlib as mp
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [28]:
iris = datasets.load_iris()
X = iris["data"][:, (2,3)] # petal lenght, petail width
y = (iris["target"] == 2).astype(np.float64) # iris virginica
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [29]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

In [30]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline({
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
})
polynomial_svm_clf.fit(X,y)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'LinearSVC(C=10, loss='hinge')' (type <class 'sklearn.svm._classes.LinearSVC'>) doesn't

In [ ]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [ ]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

In [ ]:
from sklearn.svm import LinearSVR
svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5)

In [ ]:
#support kerneltrick
from sklearn.svm import SVR
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X,y)

SVR(C=100, degree=2, kernel='poly')

In [ ]:
# Training SVM regressor on california dataset
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
#Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [39]:
from sklearn.svm import LinearSVR
lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVR(random_state=42)

In [42]:
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.09489608489057898

In [43]:
np.sqrt(mse)

0.3080520814579557

In [44]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   0.0s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   0.0s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   0.0s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   0.0s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   0.0s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   0.0s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   0.0s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   0.0s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   0.0s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   0.0s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   0.0s
[CV] END ....C=1.5808361216819946, gamma=0.05399

RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1275e2b60>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x126493430>},
                   random_state=42, verbose=2)

In [45]:
rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, gamma=0.07969454818643928)

In [46]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.2838927902562855

In [47]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.32038071669932056